In [1]:
import pandas as pd
import itertools
import yfinance as yf

from statsmodels.tsa.statespace.varmax import VARMAX
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error


In [3]:
stk_data = yf.download("RELIANCE.NS", start="2013-06-01", end="2022-02-11")
stk_data

C:\Users\admin\AppData\Local\Temp\ipykernel_17656\4010520275.py:1: FutureWarning: YF.download() has changed argument auto_adjust default to True
  stk_data = yf.download("RELIANCE.NS", start="2013-06-01", end="2022-02-11")
[*********************100%***********************]  1 of 1 completed


Price,Close,High,Low,Open,Volume
Ticker,RELIANCE.NS,RELIANCE.NS,RELIANCE.NS,RELIANCE.NS,RELIANCE.NS
Date,,,,,
2013-06-03,167.620468,171.332833,167.056686,170.981802,14165128
2013-06-04,165.982346,170.152124,165.758971,168.194884,14752690
2013-06-05,170.620148,171.109458,166.046155,166.046155,12748842
2013-06-06,168.492706,171.024351,167.663017,170.194661,17113393
2013-06-07,166.705673,170.620152,166.152542,168.960745,9420701
...,...,...,...,...,...
2022-02-04,1065.006104,1072.840798,1060.346467,1069.437362,11061241
2022-02-07,1058.519165,1076.655277,1052.991451,1069.894201,10714467


In [5]:
# performing MinMaxScaler
from sklearn.preprocessing import MinMaxScaler
MS = MinMaxScaler()
data = MS.fit_transform(stk_data)
print("Len:",data.shape)

Len: (2144, 5)


In [7]:
data = pd.DataFrame(data,columns = ["Open","High","Low","Close","Volume"])

In [9]:
data

,Open,High,Low,Close,Volume
0,0.001514,0.002707,0.004227,0.004541,0.099277
1,0.000000,0.001622,0.003019,0.001977,0.103395
2,0.004286,0.002502,0.003286,0.000000,0.089351
3,0.002320,0.002423,0.004791,0.003817,0.119940
4,0.000669,0.002052,0.003385,0.002682,0.066025
...,...,...,...,...,...
2139,0.830886,0.830910,0.835520,0.831146,0.077523
2140,0.824890,0.834414,0.828675,0.831566,0.075093
2141,0.841335,0.835757,0.830801,0.826796,0.084529
2142,0.852925,0.847173,0.851207,0.843818,0.080502


In [11]:
# Split train and test set

training_size = round(len(data) * 0.80)
print (training_size)
X_train = data[:training_size]
X_test = data[training_size:]
print ("X_train length:", X_train.shape)
print ("X_test length:", X_test.shape)
Y_train = data[:training_size]
Y_test = data[training_size:]
print ("Y_train length:", Y_train.shape)
print ("Y_test length:", Y_test.shape)

1715
X_train length: (1715, 5)
X_test length: (429, 5)
Y_train length: (1715, 5)
Y_test length: (429, 5)


In [13]:
#Define model combinations
combinations = [['Close', 'High'],['Close', 'High', 'Open'],['Close', 'High', 'Open', 'Low']]

# Result Storage
performance={"Model":[],"RMSE":[],"MaPe":[],"Lag":[],"Test":[]}


# Define exogenous variable
exog_var = ['Volume']  # make sure this exists in your dataset

In [15]:

test_obs = 28  # forecast steps

# Loop through each feature combination
for cols in combinations:
    dataset = data[cols]
    exog = data[exog_var]

    # Split data
    train = dataset[:-test_obs]
    test = dataset[-test_obs:]

    train_exog = exog[:-test_obs]
    test_exog = exog[-test_obs:]

# Find best (p, q) using AIC 
    best_aic = float('inf')  # Start with a very high AIC so anything will be better
    best_order = (0, 0)
    best_model = None

    for p in range(1, 3):  # Try AR lags from 1 to 2
        for q in range(0, 2):  # Try MA lags from 0 to 1
            try:
                model = VARMAX(train, exog=train_exog, order=(p, q), enforce_stationarity=True) # enforce_stationarity=True - ensures model stability 
                results = model.fit(disp=False)
                if results.aic < best_aic:
                    best_aic = results.aic
                    best_order = (p, q)
                    best_model = results

            except:
                continue

    # Forecast
    pred = results.forecast(steps=test_obs, exog=test_exog)
    pred_df = pd.DataFrame(pred, columns=cols)

    pred_df.to_csv("VARMAXforecasted_{}.csv".format(test_obs))

    # Metrics
    rmse = round(mean_squared_error(test, pred_df, squared=False), 4)
    mape = round(mean_absolute_percentage_error(test, pred_df), 6)

    # Save performance
    performance["Model"].append(cols)
    performance["RMSE"].append(rmse)
    performance["MaPe"].append(mape)
    performance["Lag"].append("AR=2, MA=1")
    performance["Test"].append(test_obs)

perf_df = pd.DataFrame(performance)


C:\Anaconda\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
C:\Anaconda\Lib\site-packages\statsmodels\tsa\statespace\varmax.py:160: EstimationWarning: Estimation of VARMA(p,q) models is not generically robust, due especially to identification issues.
  warn('Estimation of VARMA(p,q) models is not generically robust,'
C:\Anaconda\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
C:\Anaconda\Lib\site-packages\statsmodels\tsa\statespace\varmax.py:160: EstimationWarning: Estimation of VARMA(p,q) models is not generically robust, due especially to identification issues.
  warn('Estimation of VARMA(p,q) models is not generically robust,'
C:\Anaconda\Lib\site-packages\statsmodels\base\model.py:60

In [17]:
perf_df

,Model,RMSE,MaPe,Lag,Test
0,"[Close, High]",0.1103,0.119329,"AR=2, MA=1",28
1,"[Close, High, Open]",0.1353,0.146939,"AR=2, MA=1",28
2,"[Close, High, Open, Low]",0.1259,0.136406,"AR=2, MA=1",28


In [19]:
best_aic

-60185.113188875446

In [21]:
best_order

(2, 0)